### Checking $d_{tot} \approx w$ cases

In [1]:
from beam import *
from tests import *

In [2]:
generic_tester_Eenc(E_p_vals=[0.001,0.00001], w_vals=[20], d_tot_vals=[20], p_gph_vals=[0.3,0.7])

,E_p,w,d_tot,p_gph,err_pc_at_max,err_pc_at_min
0,0.00100,20.0,20.0,0.3,-0.002394,-0.014637
1,0.00100,20.0,20.0,0.7,-0.011648,-0.024200
2,0.00001,20.0,20.0,0.3,-0.007368,-0.020445
3,0.00001,20.0,20.0,0.7,-0.023287,-0.036832


In [3]:
generic_tester_Eenc(E_p_vals=[0.001,0.00001], w_vals=[20], d_tot_vals=[20], p_gph_vals=[0.2,0.8])

,E_p,w,d_tot,p_gph,err_pc_at_max,err_pc_at_min
0,0.00100,20.0,20.0,0.2,-0.001233,-0.010108
1,0.00100,20.0,20.0,0.8,-0.018427,-0.027400
2,0.00001,20.0,20.0,0.2,-0.004514,-0.013986
3,0.00001,20.0,20.0,0.8,-0.032690,-0.042305


In [4]:
generic_tester_Eenc(E_p_vals=[0.001,0.00001], w_vals=[30], d_tot_vals=[30], p_gph_vals=[0.2,0.8])

,E_p,w,d_tot,p_gph,err_pc_at_max,err_pc_at_min
0,0.00100,30.0,30.0,0.2,0.003971,-0.004124
1,0.00100,30.0,30.0,0.8,0.003801,-0.004386
2,0.00001,30.0,30.0,0.2,0.009475,-0.001717
3,0.00001,30.0,30.0,0.8,0.021408,0.010061


### Checking whether local minima for loss ratio crop up in the sim other than arbitrarily large/small $E_p$ vals

In [2]:
# E_p mod depth calculation code
Einput_list = [10**i for i in range(-11,13)]  #uJ
abl = 9.3  # Micromachine ablation thickness, um
width_config_list = [(2.325,abl)]  # (Filled, ablated) for 80% ablated through 0% ablated, unit:um
w_val = 34  # um
res = 25  # per um
threshold_fac = 1e-26  # Tuning factor for threshold wrt Einput, by doing this, err pc in Einput stays at the same scale for any Einput
                       # 1e-26 gives the least error a.k.a np.abs((Einput-integrate_for_energy(ee))/Einput) for res=25
alert_threshold_for_Einput = 0.0005
min_max_for_width_gap_center = []
min_max_for_width_gra_center = []

for width_config in width_config_list:
    print(f"[Info] width_config={width_config} in progress")
    tmp_loss_rat_gap = []
    tmp_loss_rat_gra = []
    for Einput in Einput_list:
        ee = beam_initialize_fast(res=res, threshold=Einput*threshold_fac, w=w_val, Ep=Einput)
        Einput_actual = integrate_for_energy(ee)
        if (Einput-integrate_for_energy(ee))/Einput > alert_threshold_for_Einput:
            print(f"[Warning] alert_threshold_for_Einput exceeded for Einput={Einput}")
        else:
            pass
        mm_gap_center = mask_initialize(beam=ee, shape='lines', width=width_config[0], thickness=width_config[1], min_max=False)
        mm_gra_center = mask_initialize(beam=ee, shape='lines', width=width_config[0], thickness=width_config[1], min_max=True)
        loss_rat_gap_center = (Einput_actual-integrate_for_energy(mask_apply(ee,mm_gap_center)))/Einput_actual
        loss_rat_gra_center = (Einput_actual-integrate_for_energy(mask_apply(ee,mm_gra_center)))/Einput_actual
        tmp_loss_rat_gap.append(loss_rat_gap_center)
        tmp_loss_rat_gra.append(loss_rat_gra_center)
    ###
    print("Data dump:")
    print(tmp_loss_rat_gap)
    print(tmp_loss_rat_gra)
    ###
    min_max_for_width_gap_center.append([min(tmp_loss_rat_gap), max(tmp_loss_rat_gap)])
    min_max_for_width_gra_center.append([min(tmp_loss_rat_gra), max(tmp_loss_rat_gra)])
#####
# Note after calcs: Even though the decrease is not monotonic, it doesn't seem that the bumps are significant, the sequences appear
# to be monotonic up to 4 sigfigs

[Info] width_config=(2.325, 9.3) in progress
Data dump:
[0.0046472647, 0.0046472824, 0.0046472405, 0.004647218, 0.004647023, 0.0046446067, 0.0046214773, 0.0044348426, 0.0038287323, 0.0033875331, 0.0032749104, 0.0032560562, 0.00325354, 0.0032531586, 0.0032531281, 0.0032530916, 0.0032530914, 0.0032530604, 0.0032530478, 0.0032531477, 0.003253036, 0.0032530741, 0.0032530539, 0.0032530373]
[0.0046472647, 0.0046472824, 0.0046472405, 0.004647218, 0.004647023, 0.0046446067, 0.0046214773, 0.0044348426, 0.0038287323, 0.0033875331, 0.0032749104, 0.0032560562, 0.00325354, 0.0032531586, 0.0032531281, 0.0032530916, 0.0032530914, 0.0032530604, 0.0032530478, 0.0032531477, 0.003253036, 0.0032530741, 0.0032530539, 0.0032530373]


In [3]:
for val in min_max_for_width_gap_center:
    ave = (min(val)+max(val))/2
    print(100*(max(val)-ave)/max(val))

15.000666836139573
